In [2]:
#basic
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from sklearn.decomposition import PCA
#Pretreatment
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
#model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
cols = [c for c in train.columns if c not in ['id', 'target','wheezy-copper-turtle-magic']]
oof_qda = np.zeros(len(train)) 
pred_qda = np.zeros(len(test))

for i in tqdm_notebook(range(512)):
    train1 = train[train['wheezy-copper-turtle-magic']==i]
    test1 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train1.index; idx2 = test1.index
    train1.reset_index(drop=True,inplace=True)
    
    data = pd.concat([pd.DataFrame(train1[cols]), pd.DataFrame(test1[cols])])
    data2 =VarianceThreshold(threshold=1.5).fit_transform(data[cols])
    train2 = data2[:train1.shape[0]]; test2 = data2[train1.shape[0]:]
    
    
    #oof
    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train1, train1['target']):

        clf = QuadraticDiscriminantAnalysis(0.1)
        clf.fit(train2[train_index,:],train1.loc[train_index]['target'])
        oof_qda[idx1[test_index]] = clf.predict_proba(train2[test_index,:])[:,1]
        pred_qda[idx2] += clf.predict_proba(test2)[:,1] / skf.n_splits
                
       
        
print('qda', roc_auc_score(train['target'], oof_svnu))


qda 0.9648973416075834
